In [51]:
import numpy as np
import seaborn as sns
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

In [16]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'

all_participants = ['VP07','VP08','VP09', 'VP10','VP11','VP12','VP13','VP14','VP18','VP19','VP20','VP22','VP23','VP24','VP25','VP26','VP27','VP28','VP29','VP30','VP31','VP32','VP33','VP35','VP36','VP37']

# path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
path = '/Users/nicobruno/ownCloud/MW_eeg_data/minmarker/' #nico-mac

In [6]:
df_markers = pd.DataFrame()

for i,v in enumerate(all_participants): 
    participant = v
    
    folder = path + participant +'/'
    
    df_ = pd.read_csv(folder+ participant + '_' + epoch_type + '_all_marker.csv', index_col = 0)
    df_['participant'] = i
    df_markers = df_markers.append(df_)
    
df_markers.to_csv('all_markers.csv')

# Univariate analysis: On-task Vs Mind- Wandering
This can only be performed for PC probes  as they are the only ones with On-task reports.

In [49]:
df_mind = (
    df_markers
    .query("probe == 'PC'")
    .assign(
    mind2 = lambda df: np.where(df.mind == 'on-task', 'on-task', 'mw')
    )
#     .query("mind != 'dMW'") #if you want to test against just one of the mw
    .drop(['participant', 'probe', 'mind'], axis = 1) # drop mind or mind2 also
)

In [50]:
markers = df_mind.drop('mind2', axis = 1).columns
for i in markers:
    auc = roc_auc_score(df_mind['mind2'], df_mind[i])
    print(f'AUC {i} = {auc}')

AUC wSMI = 0.5359910293094114
AUC p_e = 0.4982445286520764
AUC k = 0.48337328899543736
AUC b = 0.47190472507926695
AUC b_n = 0.4428273142061712
AUC g = 0.5059701492537313
AUC g_n = 0.507424019797386
AUC t = 0.5283040754775346
AUC t_n = 0.563026834738226
AUC d = 0.4825999535998763
AUC d_n = 0.4729796612790968
AUC a_n = 0.49499651999072
AUC a = 0.4967442579846879
AUC CNV = 0.5209496558657489
AUC P1 = 0.5202381873018329
AUC P3a = 0.4875957002552007
AUC P3b = 0.5717655247080659


In [29]:
label = pd.factorize(df_mind['mind2'])
feature = df_mind[].astype('float32').values

In [ ]:
# NOTE: It was 2000 in the paper. Bnut we want to save time.
n_estimators = 2000
doc_forest = make_pipeline(
    RobustScaler(),
    ExtraTreesClassifier(
        n_estimators=n_estimators, max_features=1, criterion='entropy',
        max_depth=4, random_state=42, class_weight='balanced'))

cv = GroupShuffleSplit(n_splits=50, train_size=0.8, test_size=0.2,
                       random_state=42)

aucs = cross_val_score(
    X=X, y=y, estimator=doc_forest,
    scoring='roc_auc', cv=cv, groups=np.arange(len(epochs)))

